In [1]:
import sys

In [2]:
sys.executable
# for use with
# /usr/local/opt/python@3.8/bin/python3.8 -m pip install ipython-sql

'/usr/local/opt/python@3.8/bin/python3.8'

In [3]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [4]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///chinook.db
Done.


type,name,tbl_name,rootpage,sql
table,album,album,2,"CREATE TABLE [album]( [album_id] INTEGER PRIMARY KEY NOT NULL, [title] NVARCHAR(160) NOT NULL, [artist_id] INTEGER NOT NULL, FOREIGN KEY ([artist_id]) REFERENCES [artist] ([artist_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,artist,artist,3,"CREATE TABLE [artist]( [artist_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,customer,customer,4,"CREATE TABLE [customer]( [customer_id] INTEGER PRIMARY KEY NOT NULL, [first_name] NVARCHAR(40) NOT NULL, [last_name] NVARCHAR(20) NOT NULL, [company] NVARCHAR(80), [address] NVARCHAR(70), [city] NVARCHAR(40), [state] NVARCHAR(40), [country] NVARCHAR(40), [postal_code] NVARCHAR(10), [phone] NVARCHAR(24), [fax] NVARCHAR(24), [email] NVARCHAR(60) NOT NULL, [support_rep_id] INTEGER, FOREIGN KEY ([support_rep_id]) REFERENCES [employee] ([employee_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,employee,employee,5,"CREATE TABLE [employee]( [employee_id] INTEGER PRIMARY KEY NOT NULL, [last_name] NVARCHAR(20) NOT NULL, [first_name] NVARCHAR(20) NOT NULL, [title] NVARCHAR(30), [reports_to] INTEGER, [birthdate] DATETIME, [hire_date] DATETIME, [address] NVARCHAR(70), [city] NVARCHAR(40), [state] NVARCHAR(40), [country] NVARCHAR(40), [postal_code] NVARCHAR(10), [phone] NVARCHAR(24), [fax] NVARCHAR(24), [email] NVARCHAR(60), FOREIGN KEY ([reports_to]) REFERENCES [employee] ([employee_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,genre,genre,6,"CREATE TABLE [genre]( [genre_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,invoice,invoice,7,"CREATE TABLE [invoice]( [invoice_id] INTEGER PRIMARY KEY NOT NULL, [customer_id] INTEGER NOT NULL, [invoice_date] DATETIME NOT NULL, [billing_address] NVARCHAR(70), [billing_city] NVARCHAR(40), [billing_state] NVARCHAR(40), [billing_country] NVARCHAR(40), [billing_postal_code] NVARCHAR(10), [total] NUMERIC(10,2) NOT NULL, FOREIGN KEY ([customer_id]) REFERENCES [customer] ([customer_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,invoice_line,invoice_line,8,"CREATE TABLE [invoice_line]( [invoice_line_id] INTEGER PRIMARY KEY NOT NULL, [invoice_id] INTEGER NOT NULL, [track_id] INTEGER NOT NULL, [unit_price] NUMERIC(10,2) NOT NULL, [quantity] INTEGER NOT NULL, FOREIGN KEY ([invoice_id]) REFERENCES [invoice] ([invoice_id]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([track_id]) REFERENCES [track] ([track_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,media_type,media_type,9,"CREATE TABLE [media_type]( [media_type_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,playlist,playlist,10,"CREATE TABLE [playlist]( [playlist_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,playlist_track,playlist_track,11,"CREATE TABLE [playlist_track]( [playlist_id] INTEGER NOT NULL, [track_id] INTEGER NOT NULL, CONSTRAINT [pk_playlist_track] PRIMARY KEY ([playlist_id], [track_id]), FOREIGN KEY ([playlist_id]) REFERENCES [playlist] ([playlist_id]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([track_id]) REFERENCES [track] ([track_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"


In [5]:
# Tracks by album name; subquery before the main query using a WITH clause

In [6]:
%%sql
WITH track_info AS
 (
     SELECT
        t.name as track_name,
        artist.name as artist,
        album.title album_name,
        t.milliseconds length_milliseconds,
        (t.milliseconds / 1000) length_seconds
        
     FROM track t
     INNER JOIN album ON album.album_id = t.album_id
     INNER JOIN artist ON artist.artist_id = album.artist_id
  )
  
SELECT * FROM track_info
WHERE album_name = "Ride The Lightning";

 * sqlite:///chinook.db
Done.


track_name,artist,album_name,length_milliseconds,length_seconds
Fight Fire With Fire,Metallica,Ride The Lightning,285753,285
Ride The Lightning,Metallica,Ride The Lightning,397740,397
For Whom The Bell Tolls,Metallica,Ride The Lightning,311719,311
Fade To Black,Metallica,Ride The Lightning,414824,414
Trapped Under Ice,Metallica,Ride The Lightning,244532,244
Escape,Metallica,Ride The Lightning,264359,264
Creeping Death,Metallica,Ride The Lightning,396878,396
The Call Of Ktulu,Metallica,Ride The Lightning,534883,534


In [9]:
%%sql

WITH playlist_info AS
  (
     SELECT
        playlist.playlist_id,
        playlist.name playlist_name,
        track.name track_name,
        (track.milliseconds / 1000) length_seconds
     FROM playlist
     LEFT JOIN playlist_track ON playlist_track.playlist_id = playlist.playlist_id
     LEFT JOIN track ON track.track_id = playlist_track.track_id
  )
  
SELECT
    playlist_id,
    playlist_name,
    COUNT(track_name) number_of_tracks,
    SUM(length_seconds) length_seconds
FROM playlist_info
GROUP BY 1, 2
ORDER BY 1;

 * sqlite:///chinook.db
Done.


playlist_id,playlist_name,number_of_tracks,length_seconds
1,Music,3290,876049
2,Movies,0,None
3,TV Shows,213,500987
4,Audiobooks,0,None
5,90’s Music,1477,397970
6,Audiobooks,0,None
7,Movies,0,None
8,Music,3290,876049
9,Music Videos,1,294
10,TV Shows,213,500987


In [ ]:
# Creating a custom View first; querying that view right after

In [29]:
%%sql
CREATE VIEW usa_customer_90_dollars AS
    SELECT
        customer.*
    FROM invoice
    INNER JOIN customer ON invoice.customer_id = customer.customer_id
    WHERE country = "USA"
    GROUP BY 1
    HAVING SUM(invoice.total) > 90;

SELECT * FROM usa_customer_90_dollars;


 * sqlite:///chinook.db
Done.
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4
21,Kathy,Chase,None,801 W 4th Street,Reno,NV,USA,89503,+1 (775) 223-7665,None,kachase@hotmail.com,5
22,Heather,Leacock,None,120 S Orange Ave,Orlando,FL,USA,32801,+1 (407) 999-7788,None,hleacock@gmail.com,4


In [27]:
%%sql
DROP VIEW usa_customer_90_dollars;

 * sqlite:///chinook.db
Done.


[]

In [30]:
# Where regular joins are used to join columns, the union operator is used to join rows from tables and/or views.
# Create two unique views here and make a Union for cheap americans.

In [42]:
%%sql
CREATE VIEW americans AS
    SELECT * FROM customer
    WHERE country = "USA";
    
CREATE VIEW penny_pinchers AS
    SELECT
        customer.*
    FROM invoice
    INNER JOIN customer ON invoice.customer_id = customer.customer_id
    GROUP BY 1
    HAVING SUM(invoice.total) < 20;
    
SELECT * FROM americans

UNION

SELECT * FROM penny_pinchers;

 * sqlite:///chinook.db
Done.
Done.
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
16,Frank,Harris,Google Inc.,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,+1 (650) 253-0000,+1 (650) 253-0000,fharris@google.com,4
17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
18,Michelle,Brooks,None,627 Broadway,New York,NY,USA,10012-2612,+1 (212) 221-3546,+1 (212) 221-4679,michelleb@aol.com,3
19,Tim,Goyer,Apple Inc.,1 Infinite Loop,Cupertino,CA,USA,95014,+1 (408) 996-1010,+1 (408) 996-1011,tgoyer@apple.com,3
20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4
21,Kathy,Chase,None,801 W 4th Street,Reno,NV,USA,89503,+1 (775) 223-7665,None,kachase@hotmail.com,5
22,Heather,Leacock,None,120 S Orange Ave,Orlando,FL,USA,32801,+1 (407) 999-7788,None,hleacock@gmail.com,4
23,John,Gordon,None,69 Salem Street,Boston,MA,USA,2113,+1 (617) 522-1333,None,johngordon22@yahoo.com,4
24,Frank,Ralston,None,162 E Superior Street,Chicago,IL,USA,60611,+1 (312) 332-3232,None,fralston@gmail.com,3
25,Victor,Stevens,None,319 N. Frances Street,Madison,WI,USA,53703,+1 (608) 257-0597,None,vstevens@yahoo.com,5


In [41]:
# %%sql
# DROP VIEW americans;

 * sqlite:///chinook.db
Done.


[]

In [52]:
%%sql

WITH cheap_americans AS
    (
        SELECT * FROM americans
        
        INTERSECT
        
        SELECT * FROM penny_pinchers
    )


SELECT
    employee.first_name || " " || employee.last_name employee_name,
    COUNT(cheapo.customer_id) cheap_americans
FROM employee

LEFT JOIN cheap_americans cheapo ON cheapo.support_rep_id = employee.employee_id

WHERE employee.title = "Sales Support Agent"
GROUP BY 1 ORDER BY 1;

 * sqlite:///chinook.db
Done.


employee_name,cheap_americans
Jane Peacock,0
Margaret Park,0
Steve Johnson,0


In [53]:
# with clauses allow you to define one or more named subqueries. for creating more than one named subquery. 
# To do this, you use a single WITH clause and multiple, comma-separated alias/subquery pairs:

In [54]:
# BEST CUSTOMER IN EACH COUNTRY

In [65]:
%%sql
WITH 
    customer_country_purchases AS
        (
            SELECT
                invoice.customer_id,
                customer.country,
                SUM(invoice.total) total_purchases
            FROM invoice
            INNER JOIN customer ON invoice.customer_id = customer.customer_id
            GROUP BY 1,2
        ),
    country_max_purchase AS
        (
            SELECT
                country,
                MAX(total_purchases) max_purchase
            FROM customer_country_purchases
            GROUP BY 1
        ),
    country_best_customer AS
        (
            SELECT
                country_max_purchase.country,
                country_max_purchase.max_purchase,
                (
                    SELECT customer_country_purchases.customer_id
                    FROM customer_country_purchases
                    WHERE customer_country_purchases.country = country_max_purchase.country AND country_max_purchase =
                    customer_country_purchases.total_purchases
                ) customer_id
                
            FROM country_max_purchase
        )
        
SELECT 
    country_best_customer.country country,
    customer.first_name || " " || customer.last_name customer_name,
    country_best_customer.max_purchase total_purchased
FROM customer
INNER JOIN country_best_customer ON country_best_customer.customer_id = customer.customer_id



 * sqlite:///chinook.db
(sqlite3.OperationalError) no such column: cbc.country
[SQL: WITH customer_country_purchases AS (
         SELECT
             i.customer_id,
             c.country,
             SUM(i.total) total_purchases
         FROM invoice i
         INNER JOIN customer c ON i.customer_id = c.customer_id
         GROUP BY 1, 2
        ),
    country_max_purchase AS
        (
         SELECT
             country,
             MAX(total_purchases) max_purchase
         FROM customer_country_purchases
         GROUP BY 1
        ),
    country_best_customer AS
        (
         SELECT
            cmp.country,
            cmp.max_purchase,
            (
             SELECT ccp.customer_id
             FROM customer_country_purchases ccp
             WHERE ccp.country = cmp.country AND cmp.max_purchase = ccp.total_purchases
            ) customer_id
         FROM country_max_purchase cmp
        )
SELECT
    cbc.country country,
    c.first_name || " " || c.last_name customer